In [1]:
import pandas as pd
from tqdm.auto import tqdm
import numpy as np

Скачать данные если еще не скачали

In [ ]:
import gdown

url = 'https://drive.google.com/uc?id=1wK6Z1_Nld00tIwKgwZmN2hhN9FSIOdza'
output = 'data/data.csv'  # Укажите имя выходного файла
# Скачивание файла
gdown.download(url, output, quiet=False)

url = 'https://drive.google.com/uc?id=17OhQM6PJvUtWQ5Mt-r8ACmisZ-x3S2oK'
output = 'data/post2ctr_dataset_2000.csv'  # Укажите имя выходного файла
# Скачивание файла
gdown.download(url, output, quiet=False)

url = 'https://drive.google.com/uc?id=1U2DhtkNLysxqiU2kELyM7QIvgjfisKPp'
output = 'data/post2ctr_dataset_7000.csv'  # Укажите имя выходного файла
# Скачивание файла
gdown.download(url, output, quiet=False)

url = 'https://drive.google.com/uc?id=1rDNY4tWT2g7Bzf_2ULdyRFD6FEDe2VWA'
output = 'data/post2ctr_dataset.csv'  # Укажите имя выходного файла
# Скачивание файла
gdown.download(url, output, quiet=False)

In [2]:
data = pd.read_csv("data/data.csv")
data.fillna("", inplace=True)
text_data = data[["clean_text"]]
text_data

,clean_text
0,
1,
2,новость конечно старенькая но все равно интере...
3,фантазийные бриллианты
4,сегодня на стадионе динамо прошли соревнования...
...,...
23522,тамара выиграет в 24 сезоне
23523,продажи manor lords превысили 1 млн копий сред...
23524,
23525,магическая фраза уже оплачено позвольте себе б...


In [3]:
text_data_100 = text_data.head(100)

In [4]:
from transformers import AutoTokenizer, AutoModel
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

# Загрузка модели и токенизатора
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to(device)

C:\Users\hedge\DataspellProjects\vklab_recsys\venv\lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
def extract_features(text):
    # Токенизация текста
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(device)

    # Прогон текста через модель
    with torch.no_grad():
        outputs = model(**inputs)

    # Получение эмбеддингов из последнего слоя
    embeddings = outputs.last_hidden_state.mean(dim=1)  # Среднее по токенам
    return embeddings.cpu().detach().numpy()

# Пример: извлечение фич из описания поста
tqdm.pandas()

text_data['text_feature'] = text_data['clean_text'].progress_apply(extract_features)

  0%|          | 0/23527 [00:00<?, ?it/s]

C:\Users\hedge\DataspellProjects\vklab_recsys\venv\lib\site-packages\transformers\models\xlm_roberta\modeling_xlm_roberta.py:371: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
C:\Users\hedge\AppData\Local\Temp\ipykernel_27124\1168027427.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text_data['text_feature'] = text_data['clean_text'].progress_apply(extract_features)


In [9]:
# Преобразование всех эмбеддингов в массив
embeddings_matrix = np.vstack(text_data['text_feature'].values)

# Теперь можно использовать cosine_similarity на массиве эмбеддингов
similarities = cosine_similarity(embeddings_matrix)

In [10]:
similarity_df = pd.DataFrame(similarities, columns=[f"similarity_to_{i}" for i in range(similarities.shape[1])])
data = pd.concat([data, similarity_df], axis=1)
data.head(3)

view      like   comment      hide    expand  open_photo      open  \
0  9.293762  5.225747  0.000000  1.098612  0.000000    7.574558  2.708050   
1  9.114270  5.429346  0.693147  2.079442  1.609438    6.865891  3.178054   
2  8.585412  3.258097  1.791759  2.564949  6.395262    6.066108  4.744932   

   share_to_message  \
0          3.044522   
1          1.098612   
2          1.609438   

                                                                                                                                                                                                                                                                                                                  text  \
0                                                                                                                                                                                                                                                                                                                        
1                                                                                                                                                                                                                                                                                                          • &#128571;   
2  Новость, конечно, старенькая, но все равно интересная. Около 9 месяцев назад Final анонсировали что-то планарное и очень легкое под названием X8000. О чем сегодня напомнил один из подписчиков ниже (за что ему спасибо :)) <br>И вот интересно, кто-то уже слышал? Чет я пока в продаже на гугле таких не замечал   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            